In [ ]:
!pip install -q sentence-transformers faiss-cpu groq streamlit pyngrok numpy pandas

In [ ]:
import os
import json
import faiss
from sentence_transformers import SentenceTransformer
from groq import Groq
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_path = "/content/drive/MyDrive/Classroom/DAB_RAG_ZakyProject/data/processed/academic_advisor_rag_dataset.json"

with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print("Dataset loaded:", len(data))
print("Example keys:", data[0].keys())

Dataset loaded: 5069
Example keys: dict_keys(['id', 'source_file', 'university', 'catalog_label', 'section', 'section_chunk_index', 'category', 'program', 'college', 'degree', 'level', 'text'])


In [ ]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
print("Embedding model loaded")

index_path = "/content/drive/MyDrive/Classroom/DAB_RAG_ZakyProject/rag_artifacts/academic_faiss.index"
index = faiss.read_index(index_path)

print("FAISS index loaded, vectors:", index.ntotal)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Embedding model loaded
FAISS index loaded, vectors: 5069


In [ ]:
os.environ["GROQ_API_KEY"] = "YOUR_GROQ_API_KEY"

client = Groq(api_key=os.environ["GROQ_API_KEY"])
print("Groq client initialized")


Groq client initialized


In [ ]:
%%writefile rag_engine.py
import os
import json
import faiss
from sentence_transformers import SentenceTransformer
from groq import Groq

# -------- Load dataset --------
data_path = "/content/drive/MyDrive/Classroom/DAB_RAG_ZakyProject/data/processed/academic_advisor_rag_dataset.json"
with open(data_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# -------- Embedding model --------
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# -------- FAISS index --------
index_path = "/content/drive/MyDrive/Classroom/DAB_RAG_ZakyProject/rag_artifacts/academic_faiss.index"
index = faiss.read_index(index_path)

# -------- Groq client --------
client = Groq(api_key=os.environ["GROQ_API_KEY"])

# -------- Search --------
def search(query, top_k=5):
    query_vec = model.encode([query], normalize_embeddings=True)
    scores, ids = index.search(query_vec, top_k)

    results = []
    for score, idx in zip(scores[0], ids[0]):
        r = data[int(idx)]
        results.append({
            "score": float(score),
            "text": r.get("text", ""),
            "program": r.get("program"),
            "category": r.get("category"),
            "level": r.get("level")
        })
    return results

# -------- Build context --------
def build_context(results):
    parts = []
    for i, r in enumerate(results, start=1):
        parts.append(
            f"[Source {i}] Program: {r['program']} | Category: {r['category']} | Level: {r['level']}\n{r['text']}"
        )
    return "\n\n".join(parts)

def translate_text(text, target_lang):

    system_msg = "You are a translation assistant. Translate the text accurately."
    user_msg = f"Translate this text to {target_lang}:\n{text}"

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg}
        ]
    )

    return response.choices[0].message.content.strip()

def detect_arabic(text):
    """
    Returns True if the text contains Arabic characters.
    """
    for ch in text:
        if '\u0600' <= ch <= '\u06FF':
            return True
    return False


# -------- Full RAG pipeline --------
def rag_answer_with_llm(question, top_k=3):
    results = search(question, top_k)
    context = build_context(results)

    system = (
        "You are a Smart Academic Advisor. "
        "You ONLY answer based on the provided context. "
        "If the answer does not exist, say that."
    )

    user = (
        f"Question:\n{question}\n\n"
        f"Context:\n{context}\n\n"
        "Answer concisely based only on the context."
    )

    chat = client.chat.completions.create(
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user}
        ],
        model="llama-3.1-8b-instant"
    )

    answer = chat.choices[0].message.content
    return answer, results

def smart_rag_answer(question):

    original_question = question

    # Step A: Detect language
    is_arabic = detect_arabic(question)

    # Step B: Translate to English if needed
    if is_arabic:
        question = translate_text(question, "en")

    # Step C: Get RAG answer
    answer_en, retrieved = rag_answer_with_llm(question)

    # Step D: Translate back to Arabic if input was Arabic
    if is_arabic:
        answer_ar = translate_text(answer_en, "ar")
        return answer_ar, retrieved

    # Otherwise return English answer
    return answer_en, retrieved


Overwriting rag_engine.py


In [ ]:
%%writefile app.py
import streamlit as st
from rag_engine import rag_answer_with_llm
from rag_engine import smart_rag_answer


# -----------------------
# STREAMLIT PAGE SETUP
# -----------------------
st.set_page_config(page_title="DAB - Academic Advisor", layout="wide")

st.markdown("""
<div style="text-align:center;">
    <h1>🎓 DAB</h1>
    <h3>Your smart assistant for academic questions</h3>
</div>
""", unsafe_allow_html=True)

st.markdown("""
<div style="background:#F8FAFC;padding:18px;border-radius:12px;border-left:5px solid #000;">
Ask questions about programs, courses, and requirements.
The system retrieves relevant information and provides a concise answer.
</div>
""", unsafe_allow_html=True)

# -----------------------
# USER INPUT
# -----------------------
query = st.text_input("Type your question here", placeholder="e.g. What are the requirements for Accounting?")

if st.button("🔍 Search"):
    if not query.strip():
        st.warning("Please enter a question.")
    else:
        with st.spinner("Fetching answer..."):
            try:
                # CALL RAG + LLM FUNCTION
                answer, _ = smart_rag_answer(query)
                st.subheader("🤖 Answer")
                st.success(answer)

            except Exception as e:
                st.error(f"Error generating answer: {e}")

Overwriting app.py


In [ ]:
!ngrok authtoken Your ngrok authtoken

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import subprocess, time

ngrok.kill()

# Start Streamlit
process = subprocess.Popen(
    ["streamlit", "run", "app.py", "--server.port", "8502"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

time.sleep(5)

public_url = ngrok.connect(8502)
print("🚀 Streamlit App URL:")
print(public_url)

🚀 Streamlit App URL:
NgrokTunnel: "https://uninoculative-mistie-unliterally.ngrok-free.dev" -> "http://localhost:8502"
